# FLOTVIS:
## a YOLOv4/TF-Keras based floating plastic debris detector

This code is designed to run on Google Colab <https://colab.research.google.com/>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/nschang/FLOTVIS"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


# Working on Colab

In [ ]:
# # uncomment if using Colab
# # mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# ################################
# DRIVE_PATH = '/content/drive/MyDrive'
# ################################
# import tensorflow
# # select tensorflow version
# %tensorflow_version 1.x

In [ ]:
# Google Colab Commandline upload/download
# upload
from google.colab import files
files.upload()
# download
from google.colab import files
files.download('/content/yolov4-keras-2/requirements.txt')

## Clone from Github

In [ ]:
!git clone https://github.com/nschang/FLOTVIS.git

## Download Dataset

In [ ]:
# import dataset
# also removes old dataset if present
#!rm -rf results train val VOCdevkit/VOC2007/Annotations VOCdevkit/VOC2007/JPEGImages vision_for_anchors.jpg predict.jpg prediction
LOCAL_PATH = cwd + '/VOCdevkit/VOC2007'
REMOTE_PATH = 'https://dataset.zip'

!wget -nc $LOCAL_PATH/dataset.zip $REMOTE_PATH
%cd $cwd/VOCdevkit/VOC2007/
!unzip dataset.zip
!rm -rf __MACOSX dataset.zip

# Define Variables

In [7]:
import os
################
#SAVE_PATH = 'FLOTVIS'   # name of workspace
cwd = os.getcwd()        # current working directory
LOG_PATH = 'train'       # where to save training data
RESULT_PATH = 'results'  # where to save evaluation data
################
%cd $cwd
!pwd

/Volumes/Extreme-SSD/thesis/FLOTVIS
/Volumes/Extreme-SSD/thesis/FLOTVIS


# Install Dependencies

In [ ]:
%cd $cwd
!pip install -r $cwd/requirements.txt

In [8]:
# show installed environment
!pip show tensorflow-estimator
!echo '--------'
!pip show keras
!echo '--------'
!pip show h5py
!echo '--------'
# check tensorflow version and active GPU device
import tensorflow as tf
print('tensorflow version is ', tf.__version__)
tf.test.gpu_device_name() # '/device:GPU:0' means active GPU
# get GPU Info
# !nvidia-smi

--------
Name: Keras
Version: 2.1.5
Summary: Deep Learning for humans
Home-page: https://github.com/keras-team/keras
Author: Francois Chollet
Author-email: francois.chollet@gmail.com
License: MIT
Location: /Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages
Requires: six, pyyaml, scipy, numpy
Required-by: 
--------
Name: h5py
Version: 2.10.0
Summary: Read and write HDF5 files from Python
Home-page: http://www.h5py.org
Author: Andrew Collette
Author-email: andrew.collette@gmail.com
License: BSD
Location: /Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages
Requires: numpy, six
Required-by: 
--------


ModuleNotFoundError: No module named 'tensorflow'

# --- Predict ---

In [24]:
import os
from tqdm import tqdm
image_ids1 = os.listdir('./test/')
image_ids = [f for f in os.listdir('./test/') if f.endswith(".jpg")]
print(tqdm(image_ids))
print(image_ids1)
print(image_ids)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
['.DS_Store', 'test.mp4', 'test.jpg']
['test.jpg']


# Predict

In [1]:
%cd $cwd
# get FPS
!python3 main.py --mode='fps'

[Errno 2] No such file or directory: '$cwd'
/Volumes/Extreme-SSD/thesis/FLOTVIS
Using TensorFlow backend.
/Users/nick/.pyenv/versions/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nick/.pyenv/versions/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nick/.pyenv/versions/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type

In [33]:
%cd $cwd
# predict single image
#img = input('Input image filename:')
!python3 main.py --mode='image' --img='test/test.jpg'

/Volumes/Extreme-SSD/thesis/FLOTVIS
Using TensorFlow backend.
/Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/nick/.pyenv/versions/3.6.9/envs/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qi

In [ ]:
%cd $cwd
# predict all images in folder
!python3 main.py --mode='batch' --imgdir='./test/'

In [ ]:
%cd $cwd
# predict video
!python3 main.py --mode='video' --vid='test/test.mp4'

In [ ]:
%cd $cwd
# predict using camera
!python3 main.py --mode='camera'

# --- Train ---

# Test

In [ ]:
# reload all modules before execution
%load_ext autoreload
%autoreload 2
# test
%cd $cwd/
!python test.py

# Dataset Check

In [ ]:
# simple script to evaluate the dataset integrity and consistency

# create a new folder Allempty, means to use the xml file as 
# a reference for image checking, if the image does not have 
# corresponding xml file, then move it to Allempty folder

%cd $cwd/VOCdevkit/VOC2007
import os, shutil

def checkJpgXml(dir1, dir2, dir3, is_move=True):
    """
    dir1 is the folder where the image is located
    dir2 is the folder where the annotation files are located
    dir3 is created if the image does not have a corresponding xml file, then the image is put into dir3
    is_move is to confirm whether to move or not, otherwise just print
    """
    if not os.path.exists(dir3):
        os.mkdir(dir3)
    cnt = 0
    for file in os.listdir(dir1):
        f_name,f_ext = file.split(".")
        if not os.path.exists(os.path.join(dir2, f_name+".xml")):
            print(f_name)
            if is_move:
                cnt += 1
                shutil.move(os.path.join(dir1,file), os.path.join(dir3, file))
    if cnt > 0:
        print("There are %d files that do not meet the requirements and have been printed."%(cnt))
    else:
        print("All images and corresponding xml files are one-to-one.")

if __name__ == "__main__":
    dir1 = r"JPEGImages"
    dir2 = r"Annotations"
    dir3 = r"Allempty"
    checkJpgXml(dir1, dir2, dir3, False)

In [ ]:
# number of images
!echo 'number of images'; find $cwd/VOCdevkit/VOC2007/JPEGImages -type f | wc -l
# number of labels (in .xml format)
!echo 'number of labels'; find $cwd/VOCdevkit/VOC2007/Annotations -type f | wc -l

In [ ]:
%cd $cwd/VOCdevkit/VOC2007/
!python $cwd/VOCdevkit/VOC2007/voc2yolo4.py

In [ ]:
%cd $cwd/
!python voc_annotation.py

In [ ]:
# k-means
%cd $cwd/
!python $cwd/kmeans_for_anchors.py
!mv $cwd/model_data/yolo_anchors.txt $cwd/model_data/yolo_anchors.txt.bak
!mv $cwd/yolo_anchors.txt $cwd/model_data/yolo_anchors.txt

In [ ]:
# vision for anchor
%cd $cwd/
!python ./vision_for_anchors.py

# Train

In [ ]:
# check tensorflow version and active GPU device
import tensorflow as tf
print('tensorflow version is ', tf.__version__)
tf.test.gpu_device_name() # '/device:GPU:0' means active GPU
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
# allow GPU growth
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
# Load the TensorBoard notebook extension before training
# in case of error, run: pip uninstall tensorboard-plugin-wit
%load_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir train #--host localhost --port 8088

from tensorboard import notebook
notebook.list() # View open TensorBoard instances

# Control TensorBoard display. If no port is provided, the most recently launched TensorBoard is used
#notebook.display(port=8088, height=1000) 

In [ ]:
# train
%cd $cwd
!python train.py

# Evaluate

In [ ]:
%cd $cwd
!python get_dr_txt.py

In [ ]:
!python get_gt_txt.py

In [ ]:
!python get_map.py